In [42]:
import os
import operator

with open(os.getcwd() + "/scripts/" + "Thor-Ragnarok_script.txt", "r") as s:
    thor = s.read()

In [43]:
print(thor[:300])
print("\n" + "*"*50 + "\n")
thor[:2000]




                               THOR: RAGNAROK




                                 Written by

             Eric Pearson and Craig Kyle &amp; Christopher L. Yost







 1    OMITTED                                                         1

A2   OMITTED                                           

**************************************************



"\n\n\n                               THOR: RAGNAROK\n\n\n\n\n                                 Written by\n\n             Eric Pearson and Craig Kyle &amp; Christopher L. Yost\n\n\n\n\n\n\n\n 1    OMITTED                                                         1\n\nA2   OMITTED                                                         A2\n\nA3   THE MARVEL LOGO. SMOLDERING, BEGINNING TO TURN ORANGE IN THE    A3\n     HEAT AS WE TILT UP TO SEE-\n\n     -FIRE.    NOTHING BUT FIRE.\n\n2    INT. TIGHT SPACE - INDETERMINATE TIME                           2\n\n     Dark and cramped. The soft red light of fire seeps through\n     iron slats. Inside this cage is a man, bound by chains.\n\n     It's THOR. His beard is long and his clothes are worn. That\n     rough, grizzled look of a man who's spent years on the road.\n\n     He awakens with a JOLT.   Looks around.\n\n                           THOR\n                 Now I know what you're thinking.\n                 Oh no! Thor's in a cage. How

In [44]:
# Gets the frecuency of appearance of spaces at the beginning
# (0, 2335) means that 2335 lines begin with 0 spaces
line_space_freq = {}
for line in thor.split("\n"):
    key = line.count(" ")
    if key not in line_space_freq:
        line_space_freq[key] = 1
    else:
        line_space_freq[key] += 1
sorted_line_space_freq = sorted(line_space_freq.items(), key=operator.itemgetter(1), reverse=True)
sorted_line_space_freq[:10]

[(0, 2335),
 (15, 437),
 (20, 426),
 (16, 375),
 (14, 318),
 (21, 313),
 (13, 241),
 (10, 229),
 (17, 228),
 (11, 227)]

0 corresponds to screen-play and randoms `""`. 1 to 5 or so corresponds also to screen-play. Below 17 corresponds to character dialogue. Above corresponds to names and scenes and stuff introductions. Thus, a space of 10 to cut is OK, at least for this movie.

In [45]:
# Counts the frequency of every unique sentence that starts with more than 10 spaces
# With the spaces before and after text removed, and also with "(some_text)" removed 
poss_char = {}
for line in thor.split("\n"):
    if line.count(" ") > 10:  # Relative low space (this condition may not even be neccessary)
        # Need to do some processing
        key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
        if key:  # To omit ""
            if key not in poss_char:
                poss_char[key] = 1
            else:
                poss_char[key] += 1

In [46]:
# Gets sentences with more than 5 occurences (potential characters)
sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
threshold = 5
chars = []
for char_tuple in sorted_poss_chars:
    if char_tuple[1] > threshold:
        chars.append(char_tuple)
    else:
        break
chars

[('THOR', 348),
 ('LOKI', 88),
 ('VALKYRIE', 71),
 ('BANNER', 64),
 ('HELA', 59),
 ('GRANDMASTER', 49),
 ('HULK', 42),
 ('KORG', 33),
 ('ODIN', 30),
 ('SKURGE', 19),
 ('HEIMDALL', 19),
 ('SURTUR', 14),
 ('SCRAPPER #142', 14),
 ('STRANGE', 13),
 ('ACTOR THOR', 12),
 ('ACTOR LOKI', 11),
 ('TOPAZ', 10),
 ('QUINJET', 9),
 ('SOOTHING VOICE', 8),
 ('DR. STRANGE', 7)]

#### **Idea**:

This threshold (5 in this case) could be tuned using IMDb! We compare with the list of main characters listed on site, get the overall accuracy for all movies for different thresholds, and choose the threshold with the highest accuracy. 

*UPDATE*: Upon much thought, we will choose the characters with the most dialogue instead. TODO: Decide on the number of characters....

Note: the `if line.count(" ") > 10:` condition may not be needed, which would be great for those scripts that do not follow this format. See below:

In [47]:
poss_char = {}
for line in thor.split("\n"):
    key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
    if key:
        if key not in poss_char:
            poss_char[key] = 1
        else:
            poss_char[key] += 1

In [48]:
sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
threshold = 5
chars = []
for char_tuple in sorted_poss_chars:
    if char_tuple[1] > threshold:
        chars.append(char_tuple)
    else:
        break
chars

[('THOR', 357),
 ('LOKI', 88),
 ('VALKYRIE', 71),
 ('BANNER', 67),
 ('HELA', 59),
 ('GRANDMASTER', 49),
 ('HULK', 42),
 ('KORG', 33),
 ('ODIN', 30),
 ('SKURGE', 19),
 ('HEIMDALL', 19),
 ('STRANGE', 17),
 ('SURTUR', 14),
 ('SCRAPPER #142', 14),
 ('ACTOR THOR', 12),
 ('ACTOR LOKI', 11),
 ('TOPAZ', 10),
 ('QUINJET', 9),
 ('SOOTHING VOICE', 8),
 ('What?', 8),
 ('DR. STRANGE', 7)]

The result is pretty much the same, although `'What?'` is apparantly a character. This could be avoided by `.isupper()` or maybe checking for punct sings. The `x` in `if line.count(" ") > x` could also be tuned.  

Actually, I found this to be problematic, so I set it to `-1`.

In [49]:
with open(os.getcwd() + "/scripts/" + "Braveheart_script.txt", "r") as s:
    braveheart = s.read()

line_space_freq = {}
for line in braveheart.split("\n"):
    key = line.count(" ")
    if key not in line_space_freq:
        line_space_freq[key] = 1
    else:
        line_space_freq[key] += 1
sorted_line_space_freq = sorted(line_space_freq.items(), key=operator.itemgetter(1), reverse=True)
sorted_line_space_freq[:10]

[(0, 1634),
 (37, 638),
 (25, 511),
 (26, 455),
 (27, 302),
 (24, 294),
 (32, 293),
 (31, 279),
 (30, 190),
 (23, 150)]

In [50]:
poss_char = {}
for line in braveheart.split("\n"):
    if line.count(" ") > 10:
        key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
        if key:
            if key not in poss_char:
                poss_char[key] = 1
            else:
                poss_char[key] += 1

sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
threshold = 5
chars = []
for char_tuple in sorted_poss_chars:
    if char_tuple[1] > threshold:
        chars.append(char_tuple)
    else:
        break
chars

[('WALLACE', 114),
 ('PRINCESS', 63),
 ('WILLIAM', 56),
 ('HAMISH', 38),
 ('ROBERT', 37),
 ('LONGSHANKS', 34),
 ('EDWARD', 30),
 ('CRAIG', 26),
 ('CAMPBELL', 23),
 ('MAGISTRATE', 22),
 ('STEPHEN', 20),
 ('MARION', 18),
 ('ARGYLE', 15),
 ('VOICE OVER', 11),
 ('MORNAY', 11),
 ('TALMADGE', 9),
 ('FATHER', 7),
 ('THE LEPER', 7),
 ('LOCHLAN', 7),
 ('SOLDIER', 6),
 ('NICOLETTE', 6)]

In [51]:
def get_characters(script, threshold_char=5, threshold_space=-1):
    
    with open(os.getcwd() + "/scripts/" + script, "r") as s:
        script = s.read()
        
    poss_char = {}
    for line in script.split("\n"):
        if line.count(" ") > threshold_space:
            key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ").replace("\t", "").strip(" ")
            if key not in poss_char:
                poss_char[key] = 1
            else:
                poss_char[key] += 1
                        
    sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
    chars = []
    for char_tuple in sorted_poss_chars:
        if char_tuple[1] > threshold_char:
            chars.append(char_tuple)
        else:
            break
            
    return [char[0] for char in chars if char[0] != "" and "-" not in char[0] and "." not in char[0] and ":" not in char[0] and "?" not in char[0]]


get_characters("Thor-Ragnarok_script.txt")

['THOR',
 'LOKI',
 'VALKYRIE',
 'BANNER',
 'HELA',
 'GRANDMASTER',
 'HULK',
 'KORG',
 'ODIN',
 'SKURGE',
 'HEIMDALL',
 'STRANGE',
 'SURTUR',
 'SCRAPPER #142',
 'ACTOR THOR',
 'ACTOR LOKI',
 'TOPAZ',
 'QUINJET',
 'SOOTHING VOICE']

In [70]:
problematic_scripts = []
for script in os.listdir(os.getcwd() + "/scripts/"):
    if script[-4:] == ".txt":
        chars = get_characters(script)
        if len(chars) == 0:
            problematic_scripts.append(script)
            brr = True
        else:
            pass

In [71]:
set(problematic_scripts)

{'Aladdin_script.txt',
 'American-Outlaws_script.txt',
 'Artist,-The_script.txt',
 'Dawn-of-the-Dead_script.txt',
 'Legend_script.txt',
 'Made_script.txt',
 'Mulan_script.txt',
 'Training-Day_script.txt'}

In [72]:
len(set(problematic_scripts))

8

In [54]:
import requests
from bs4 import BeautifulSoup
dict_lens = {}
for i in set(problematic_scripts):
    website_url = requests.get("https://www.imsdb.com/scripts/" + i[:i.find("_")] + ".html").text
    soup = BeautifulSoup(website_url, "html.parser")
    if soup.pre.find_all("pre"):
        script = ""
        for j in soup.pre.find_all("pre"):
            script += str(j)
    else:
        script = str(soup.pre)
    dict_lens[i] = len(script)

In [55]:
dict_lens

{'Legend_script.txt': 155467,
 'Red-Planet_script.txt': 1577,
 'Equilibrium_script.txt': 1950,
 "Monster's-Ball_script.txt": 1730,
 'Artist,-The_script.txt': 100324,
 'Courage-Under-Fire_script.txt': 2022,
 'Jade_script.txt': 1660,
 'Mulan_script.txt': 95600,
 'At-First-Sight_script.txt': 1581,
 'Aladdin_script.txt': 106929,
 'What-Women-Want_script.txt': 1806,
 'Speed_script.txt': 1918,
 'True-Romance_script.txt': 1579,
 'Blues-Brothers,-The_script.txt': 2025,
 'Made_script.txt': 198436,
 'American-Outlaws_script.txt': 219461,
 'Saint,-The_script.txt': 1576,
 'Dawn-of-the-Dead_script.txt': 235774,
 'Lion-King,-The_script.txt': 1985,
 'Barbershop_script.txt': 1958,
 'Back-to-the-Future_script.txt': 2022,
 'Training-Day_script.txt': 198900,
 "Mr.-Holland's-Opus_script.txt": 1758}

In [31]:
website_url = requests.get("https://www.imsdb.com/scripts/" + "Lion-King,-The" + ".html").text
soup = BeautifulSoup(website_url, "html.parser")
if soup.pre.find_all("pre"):
    script = ""
    for i in soup.pre.find_all("pre"):
        script += str(i)
else:
    script = str(soup.pre)
script = script.replace("<pre>", "").replace("</pre>", "").replace("<b>", "")
script = script.replace("</b>", "").replace("<u>", "").replace("</u>", "")
print(script[:1000])

 <html>
<head>
<title>Lion King, The Script at IMSDb.</title>
<meta content="Lion King, The script" name="description"/>
<meta content="Lion King, The script, Lion King, The movie script, Lion King, The film script" name="keywords"/>
<p align="center">
<iframe frameborder="0" height="90" scrolling="no" src="http://www.imsdb.com/adbanner.html" width="728"></iframe>
<br/><br/>
This script was brought to you by :<br/>
  <a href="http://www.imsdb.com">The Internet Movie Script Database</a><br/>
  www.IMSDb.com
</p><table cellpadding="0" cellspacing="0" width="99%">
<tr>
<td valign="top" width="120">

<iframe border="0" frameborder="0" height="600" marginheight="0" marginwidth="0" scrolling="no" src="http://www.filemania.com/ad_sky.html" style="border:none;" width="120"></iframe>

<td><td valign="top" width="120">

<iframe border="0" frameborder="0" height="240" marginheight="0" marginwidth="0" scrolling="no" src="http://rcm.amazon.com/e/cm?t=everlongsoftw-20&amp;o=1&amp;p=8&amp;l=st1&amp;m

In [33]:
len(script)

1974

In [32]:
"\n" in script

True

In [24]:
with open(os.getcwd() + "/scripts/" + 'Training-Day_script.txt', "r") as s:
    script = s.read()

print(script[:1000])

                             <u>TRAINING DAY</u>                                  by                              David Ayer                                              <u>REVISED DRAFT</u>                                              August 18, 1999        FADE IN:        INT. BEDROOM - PREDAWN        In a Simi Valley housing tract.  It's dark.        CLOSE ON ALARM CLOCK        4:59 AM.        WIDER ANGLE:  Eyes glisten in the clock's shine,        patiently watching the time.        BZZZZ!  The ALARM SOUNDS.  A hand that's been hovering        over the clock drops.  Silence.  JAKE HOYT, a fit young        man, rolls over, throws an arm over his sleeping wife --                                JAKE                  It's time.        -- she's not there, just empty blankets.  He sits up.        HIS POV        LISA, his wife, in a rocking chair in the corner.        BACK TO SCENE        Jake is surprised.  How'd she sneak off?                                JAKE                  What ar

These appear to be the `html` scripts.

There are many more than I though... TODO: Fix this!!!!

In [230]:
import requests
from bs4 import BeautifulSoup

website_url = requests.get("https://www.imsdb.com/scripts/" + "Alien" + ".html").text
soup = BeautifulSoup(website_url, "html.parser")
script = str(soup.pre.pre)

In [231]:
"\n" in script

True

In [272]:
script = script.replace("<pre>", "").replace("</pre>", "").replace("<b>", "").replace("</b>", "").replace("</u>", "").replace("<u>", "")
print(script[4000:6000])



     CAMERA ANGLES GRADUALLY WIDEN, revealing more and more of the
     machinery, banks of panels, fluttering gauges, until we reveal:

     INTERIOR - HYPERSLEEP VAULT

     A stainless steel room with no windows, the walls packed with
     instrumentation.  The lights are dim and the air is frigid.

     Occupying most of the floor space are rows of horizontal FREEZER
     COMPARTMENTS, looking for all the world like meat lockers.

     FOOM!  FOOM!  FOOM!  With explosions of escaping gas, the lids on the
     freezers pop open.

     Slowly, groggily, six nude men sit up.

                              ROBY
               Oh... God... am I cold... 

                              BROUSSARD
               Is that you, Roby?

                              ROBY
               I feel like shit... 

                              BROUSSARD
               Yeah, it's you all right.

     Now they are yawning, stretching, and shivering.

                              FAUST
                

In [282]:
poss_char = {}
for line in script.split("\n"):
    key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ").replace("\t", "").strip(" ")
        #print(key)
    if key not in poss_char:
        poss_char[key] = 1
    else:
        poss_char[key] += 1

sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
chars = []
for char_tuple in sorted_poss_chars:
    if char_tuple[1] > 5:
        chars.append(char_tuple)
    else:
        break
        
#print(sorted_poss_chars)

chars = [char[0] for char in chars if char[0] != "" and "-" not in char[0] and "." not in char[0] and ":" not in char[0]]
chars

['STANDARD', 'ROBY', 'HUNTER', 'MELKONIS', 'BROUSSARD', 'FAUST', 'COMPUTER']

In [320]:
# Taken from script_processor.py and slightly modified
scripts_char_diags = {}
#create a string containing character names seperated by commas
characters = ','.join([','.join(name.split(' ')) for name in chars if len(name.split(' ')) <3])
#print(characters)
# data structure to store the dialogues
dialogues = {}
for ch in characters.split(','):
    #print(ch)
    if 'Mr.' in ch or '-' in ch:
        continue
    else:
        name = ch
    dialogue = '' 
    flag = 0
    count = 0
    for line in script.split("\n"):
        temp = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
        if temp.isupper():  # If Character Intro
            if name.upper() == temp:
                flag = 1
                count += 1
                dialogue += '\n['+str(count)+']'
                continue
            else:  # If it's another character
                flag = 0
                continue
        if flag:
            if flag == 1:
                if line.lstrip(" ")[0] == "(" or line.rstrip(" ")[-1] == ")":  # To still get the dialogue
                    ident_level_first = None  # when the first line below a character
                    flag = 0  # intro is something like "(looking around with a grin)"
                else:  # which describes how the character is acting but it is not his dialogue
                    ident_level_first = len(line) - len(line.lstrip(" "))
            ident_level = len(line) - len(line.lstrip(" "))
            if ident_level_first:
                if ident_level == ident_level_first:
                    dialogue += temp + " "
                else:
                    continue
            else:
                pass
            flag += 1
    dialogues[name] = dialogue
scripts_char_diags['dialogues'] = dialogues

In [256]:
script

''

In [250]:
if soup.pre.find_all("pre"):
    print("m")

In [227]:
dict_lens

{'Made_script.txt': 198645,
 'Back-to-the-Future_script.txt': 2022,
 'Alien_script.txt': 203578,
 'Dawn-of-the-Dead_script.txt': 235774,
 'Speed_script.txt': 1918,
 'Jade_script.txt': 1660,
 "Mr.-Holland's-Opus_script.txt": 1758,
 'Lion-King,-The_script.txt': 1985,
 "Monster's-Ball_script.txt": 1730,
 'Legend_script.txt': 155467,
 'Courage-Under-Fire_script.txt': 2022,
 'American-Outlaws_script.txt': 219702,
 'True-Romance_script.txt': 1579,
 'Mulan_script.txt': 95600,
 'Aladdin_script.txt': 106929,
 'Training-Day_script.txt': 199002,
 'What-Women-Want_script.txt': 1806,
 'Saint,-The_script.txt': 1576,
 'Equilibrium_script.txt': 1950,
 'Blues-Brothers,-The_script.txt': 2025,
 'At-First-Sight_script.txt': 1581,
 'Artist,-The_script.txt': 100324,
 'Barbershop_script.txt': 1958,
 'Red-Planet_script.txt': 1577}

In [216]:
def get_characters(script, threshold_char, threshold_space=-1):
    
    with open(os.getcwd() + "/scripts/" + script, "r") as s:
        script = s.read()
    print(script[:6000])
        
    poss_char = {}
    for line in script.split("\n"):
        if line.count(" ") > threshold_space:
            key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ").replace("\t", "").strip(" ")
                #print(key)
            if key not in poss_char:
                poss_char[key] = 1
            else:
                poss_char[key] += 1
                        
    sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
    chars = []
    for char_tuple in sorted_poss_chars:
        if char_tuple[1] > threshold_char:
            chars.append(char_tuple)
        else:
            break
            
    #print(sorted_poss_chars)
                
    #processed_names = [char[0].replace("\t", "").strip() for char in chars]  # See Raging-Bull_script.txt
    return [char[0] for char in chars if char[0] != ""]

In [229]:
get_characters("Alien_script.txt")

[]

In [124]:
s = 0
for line in scriptt.split("\n"):
    if line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ").replace("\t", "").strip(" ") == "JOHNNY":
        s += 1
s

286

In [219]:
with open(os.getcwd() + "/scripts/" + "Starship-Troopers_script.txt", "r") as s:
    scriptt = s.read()
#print(scriptt[:10000])

In [220]:
#print(script.replace("<pre>", "").replace("</pre>", "").replace("<b>", "").replace("</b>", "")[:2000])

Upon much thought, we will choose the characters with the most dialogue instead. TODO: Decide on the number of characters....

In [11]:
# Taken from `script_processor.py`

import omdb
import requests
from bs4 import BeautifulSoup

files = os.listdir(os.getcwd() + '/scripts/')

#Create a a dict where each item contains information on a movie in dictionary format
movies = {}
for each in files:
    movie = {}
    if '-The_script.txt' in each:
        movies['The '+each.replace('-The_script.txt','').replace('-', ' ')[:-1]] = {'filename':each}
    else:
        movies[each.replace('_script.txt','').replace('-', ' ')] = {'filename':each}

#fetch imdb data of a movie using OMDB api, please read OMDB doc for further details
def getOmdbData(movie):
    api_key = 'eb6547ff'
    omdb.set_default('apikey', api_key)
    return omdb.get(title=movie, fullplot=True, tomatoes=True)

def prepURL(imdb_id):
	#prepare the url from which we will scrape the cast data
    url = 'https://www.imdb.com/title/'+imdb_id#+'/fullcredits?ref_=tt_cl_sm#cast'
    return url

def getCharacters(url):
	#Only first imdb page of the cast members are considered. 
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url, "lxml")
    #upon inspecting the html code, we see that the cast is structured in <table> tags
    table = soup.find('table',{'class':'cast_list'}) 
    #<tr> is a tag for table row [tr]
    cast = table.find_all('tr') 
    characters = []
    for each in cast:
        temp = each.find_all('td')[-1].text.strip().replace('\n','').replace('  ','')
        if 'uncredited' in temp:
            characters.append(temp.split('(')[0][:-1])
        else:
            characters.append(temp)
    characters = list(set(characters))
    return characters

for movie in movies.keys():
	#for the given movie name we fetch the imdb data
    x = getOmdbData(movie)
    try:
        movies[movie]['imdb_id'] = x['imdb_id']
    except:
        continue

#We need to delete the movie's on which there isn't much data available
delete = []
for movie in movies.keys():
    try:
        url = prepURL(movies[movie]['imdb_id'])
        characters = getCharacters(url)
        movies[movie]['imdb_url'] = url
        movies[movie]['characters'] = characters
    except:
        delete.append(movie)

#Delete the movies on which we don't have necessary data on
for movie in delete:
    del movies[movie]

#Clean the data further
for movie in movies.keys():
    if 'Cast overview, first billed only:' in movies[movie]['characters']:
        movies[movie]['characters'].remove('Cast overview, first billed only:')

In [29]:
# Taken from script_processor.py and slightly modified
scripts_char_diags = {}
for script in os.listdir(os.getcwd() + "/scripts/"):
    scripts_char_diags[script] = {}
    if script[-4:] == ".txt":
        chars = get_characters(script, 5)  
        with open(os.getcwd() + "/scripts/" + script, "r") as s:
            scriptt = s.read()
        #create a string containing character names seperated by commas
        characters = ','.join([','.join(name.split(' ')) for name in chars if len(name.split(' ')) <3])
        # data structure to store the dialogues
        dialogues = {}
        for ch in characters.split(','):
            if 'Mr.' in ch or '-' in ch:
                continue
            else:
                name = ch
            dialogue = '' 
            flag = 0
            count = 0
            for line in scriptt.split("\n"):
                temp = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
                if temp.isupper():  # If Character Intro
                    if name.upper() in temp:
                        flag = 1
                        count += 1
                        dialogue += '\n['+str(count)+']'
                        continue
                    else:  # If it's another character
                        flag = 0
                        continue
                if flag:
                    if flag == 1:
                        ident_level_first = len(line) - len(line.lstrip(" "))
                    ident_level = len(line) - len(line.lstrip(" "))
                    if ident_level == ident_level_first:
                        dialogue += temp + " "
                    else:
                        continue
                    flag += 1
            dialogues[name] = dialogue
        scripts_char_diags[script]['dialogues'] = dialogues

In [54]:
dict_lengths = {}
for key, value in scripts_char_diags["Thor-Ragnarok_script.txt"]["dialogues"].items():
    a = value.replace("\n", "").replace(" ", "")
    i = 1
    while True:
        before = len(a)
        a = a.replace("[" + str(i) + "]", "")
        i += 1
        if len(a) == before:
            break
    dict_lengths[key] = len(a)

In [55]:
sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)

[('THOR', 14805),
 ('LOKI', 3757),
 ('GRANDMASTER', 3266),
 ('HELA', 2801),
 ('VALKYRIE', 2507),
 ('KORG', 2430),
 ('BANNER', 2319),
 ('ODIN', 1751),
 ('SKURGE', 1109),
 ('ACTOR', 812),
 ('STRANGE', 752),
 ('SURTUR', 666),
 ('SOOTHING', 639),
 ('VOICE', 639),
 ('HEIMDALL', 633),
 ('HULK', 615),
 ('SCRAPPER', 440),
 ('#142', 361),
 ('TOPAZ', 267),
 ('DR.', 172),
 ('QUINJET', 160)]

In [56]:
[i[0] for i in sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)[:5]]

['THOR', 'LOKI', 'GRANDMASTER', 'HELA', 'VALKYRIE']

In [60]:
overall_acc = {}
problematic_scripts = []
print_one = True
n_characters_choose = 5
accs = []
for script in os.listdir(os.getcwd() + "/scripts/"):
    if script[-4:] == ".txt":
        # Gets the characters using our function
        chars = get_characters(script, 5)
        if len(chars) == 0:
            problematic_scripts.append(script)
            brr = True
        else:
            # Gets the top n_characters_choose characters with the most dialogue
            dict_lengths = {}
            for key, value in scripts_char_diags[script]["dialogues"].items():
                a = value.replace("\n", "").replace(" ", "")
                i = 1
                while True:
                    before = len(a)
                    a = a.replace("[" + str(i) + "]", "")
                    i += 1
                    if len(a) == before:
                        break
                dict_lengths[key] = len(a)
            chars_sorted_len_diag = sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)
            chars = [i[0] for i in chars_sorted_len_diag[:n_characters_choose]]
            #Gets list of characters from IMDB
            if '-The_script.txt' in script:
                key = 'The '+script.replace('-The_script.txt','').replace('-', ' ')[:-1]
            else:
                key = script.replace('_script.txt','').replace('-', ' ')
            try:
                chars_imdb = movies[key]["characters"]
            except KeyError:
                pass
            else:
                #Turns both to UPPERCASE (in case chars is not, and IMDB will most likely not be)
                chars_upper = [char.upper() for char in chars]
                chars_imdb_upper = [char.upper() for char in chars_imdb]
                # Compares one by one and add 1 to `matches` if at least one word of one is equal to one word of the other?
                # This condition could probably be improved, but we need to do some visual testing, so we probably want to
                # keep a list with the names of the characters that matched
                matches = 0
                br = False
                for imdb_char in chars_imdb_upper:
                    for word in imdb_char.split(" "):
                        for char in chars_upper:
                            if word in char:
                                matches += 1
                                br = True
                                break
                        if br:
                            break
                accs.append(matches/len(chars))  # Proportion of identified characters that were in IMDb's main cast
                if accs[-1] < 0.1 and print_one:
                    print(script)
                    print_one = False

print(sum(accs)/len(accs))

Godzilla_script.txt
0.637686139747997


In [61]:
movies["Godzilla"]["characters"]

['Vivienne Graham',
 'Joe Brody',
 'Captain Russell Hampton',
 'Jump Master',
 'Admiral William Stenz',
 'Sergeant Tre Morales',
 'HALO Jumper',
 'Young Ford',
 'Elle Brody',
 'Sandra Brody',
 'Dr. Ishiro Serizawa',
 'Sam Brody',
 'Bomb Tracker',
 'Lieutenant Commander Marcus Waltz',
 'Ford Brody']

In [62]:
get_characters("Godzilla_script.txt", 5)

['NICK',
 'AUDREY',
 'RAYMOND',
 'MAJOR HICKS',
 'ANIMAL',
 'CAIMAN',
 'LUCY',
 'MAYOR',
 'CUT TO:',
 'ELSIE',
 "SERGEANT O'NEAL",
 'GENERAL ANDERSON',
 'CAPTAIN',
 'JEAN-CLAUDE',
 'ENSIGN']

I think it's actually [Godzilla from 1998](https://www.imdb.com/title/tt0120685/)... We may be better off using my function and we can avoid issues such as this one. For a better evaluation, I think Sean was using the release date on our `.csv` to get the correct movie, when there are these discrepancies.

In [63]:
with open(os.getcwd() + "/scripts/" + "Godzilla_script.txt", "r") as s:
    scriptt = s.read()

In [26]:
import os
import json


def get_characters(script, threshold_char=5, threshold_space=-1):
    
    try:
        with open(os.getcwd() + "/scripts/" + script, "r") as s:
            script = s.read()
    except Exception as e:
        print(e)
        print(script)
        
    else:
        
        poss_char = {}
        for line in script.split("\n"):
            if line.count(" ") > threshold_space:
                key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ").replace("\t", "").strip(" ")
                if key not in poss_char:
                    poss_char[key] = 1
                else:
                    poss_char[key] += 1

        sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
        chars = []
        for char_tuple in sorted_poss_chars:
            if char_tuple[1] > threshold_char:
                chars.append(char_tuple)
            else:
                break
            
    return [char[0] for char in chars if char[0] != "" and "-" not in char[0] and "." not in char[0] and ":" not in char[0] and "?" not in char[0]]


#Alternate Approach which works equally well.
"""
script = open('NLP-Movie_Scripts/scripts/'+file,'r')
for line in script:
    temp = line.strip()
    if 'thor'.upper() in temp:
        if (len(temp.split(' '))  <= 2) & (temp.split(' ')[0]=='thor'.upper()):
            print(temp)
"""
#lns = []
movies = {}
for movie in os.listdir(os.getcwd() + "/scripts/"):
    movies[movie] = {}
    if movie[-4:] == ".txt":
        with open(os.getcwd() + "/scripts/" + movie, "r") as s:
            script = s.read()
        #create a string containing character names seperated by commas
        characters = ','.join([','.join(name.split(' ')) for name in get_characters(movie) if len(name.split(' ')) <3])
        # data structure to store the dialogues
        dialogues = {}
        for ch in characters.split(','):
            if 'Mr.' in ch or '-' in ch:
                continue
            else:
                name = ch
            dialogue = '' 
            flag = 0
            count = 0
            for line in script.split("\n"):
                temp = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
                if temp.isupper():  # If Character Intro
                    if name.upper() == temp:
                        flag = 1
                        count += 1
                        dialogue += '\n['+str(count)+']'
                        continue
                    else:  # If it's another character
                        flag = 0
                        continue
                if flag:
                    if flag == 1:
                        #lns.append(line)
                        #print(line)
                        #print(ch)
                        if line.strip(): # Accounts for first lines == "" or == "     "
                            if line.lstrip(" ")[0] == "(" or line.rstrip(" ")[-1] == ")":  # To still get the dialogue
                                ident_level_first = None  # when the first line below a character
                                flag = 0  # intro is something like "(looking around with a grin)"
                            else:  # which describes how the character is acting but it is not his dialogue
                                ident_level_first = len(line) - len(line.lstrip(" "))
                        else:
                            ident_level_first = None
                            flag = 0
                    ident_level = len(line) - len(line.lstrip(" "))
                    if ident_level_first:
                        if ident_level == ident_level_first:
                            dialogue += temp + " "
                        else:
                            continue
                    else:
                        pass
                    flag += 1
            dialogues[name] = dialogue
        movies[movie]['dialogues'] = dialogues

In [41]:
movies["Assassins_script.txt"]["dialogues"]["KETCHAM"][:4000]

"\n[1]Aw-shit! \n[2]How much farther? \n[3]Oh, fuck it. \n[4]Hey, this feels kinda good. \n[5]Hey, do you mind if I talk a little?  I feel like, I don't know, talking I guess. \n[6]Funny, I've never been a talker. My wife was always getting on me about that.  'Say what you feel, tell me what's bothering you, you ve got to talk to me.'  I never would though.  Not really. \n[7]I don't know.  Part of me wanted to but part of me always said, 'What's she going to be able to do?'  I don't know.  Maybe I didn't trust her. \n[8]My Margaret... I loved her but you know what?  I cheated on her.  All the time.  I don't even know why. It wasn't the sex, Margaret was great in bed.  I think if I wasn't married I wouldn't have even looked at some of those women.  I knew she couldn't trust me, so how could I trust her?  If she was here right now I'd think that I would tell her that I was sorry... 'course, if she were here right now, maybe I wouldn't feel like talking. \n[9]I think I've heard of you. \n

In [50]:
lns[-1]

'                         '

In [57]:
with open(os.getcwd() + "/scripts/" "Lion-King,-The_script.txt", "r") as s:
    script = s.read()
len(script)

1974

In [60]:
ready_scripts[0]

'Wolf-of-Wall-Street,-The'

In [75]:
ready_scripts = [file_name[:file_name.find("_")] for file_name
                 in os.listdir(os.getcwd() + "/scripts/")
                 if ".txt" in file_name]
n_scripts_before = len(ready_scripts)

for script in ready_scripts:
    path = os.getcwd() + "/scripts/" + script + "_script.txt"
    with open(path, "r") as s:
        m = s.read()
    # Deletes scripts that were pdf and thus we got html trash
    if len(m) < 3000:
        os.remove(path)
        continue
    # Deletes scripts that do not follow the standard format
    if len(get_characters(script + "_script.txt")) == 0:
        os.remove(path)
        
ready_scripts = [file_name[:file_name.find("_")] for file_name
                 in os.listdir(os.getcwd() + "/scripts/")
                 if ".txt" in file_name]
print("We removed", len(ready_scripts) - len(n_scripts_before), "scripts")

In [76]:
len(os.listdir())

14

In [77]:
ready_scripts = [file_name[:file_name.find("_")] for file_name
                 in os.listdir(os.getcwd() + "/scripts/")
                 if ".txt" in file_name]
len(ready_scripts)

680